# The program is at bottom mentoned as "Automating publish dates - final code" this is the code that is used to scrap the given website


## The output is saved in 3 different varients:
## 1. As pandas dataframe
## 2. As excel file
## 3. As csv file

In [1]:
!pip install requests beautifulsoup4 pandas



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


## Final code

In [103]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Initialize the WebDriver and load the page
driver = webdriver.Chrome()
url = 'https://journals.sagepub.com/toc/JMX/current'
driver.get(url)

# Allow the page to load completely
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'table-of-content')))

# Find the div with the class 'table-of-content'
content_div = driver.find_element(By.CLASS_NAME, 'table-of-content')

# Find all section tags within this div
sections = content_div.find_elements(By.TAG_NAME, 'section')

# List to store extracted data
data = []

# Iterate over each section
for section in sections:
    try:
        # Locate the "Preview abstract" button using its attributes
        preview_button = WebDriverWait(section, 10).until(
            EC.element_to_be_clickable((By.XPATH, ".//button[@data-toggle='collapse' and contains(@class, 'btn-link')]"))
        )
        preview_button.click()
        
        # Wait for the abstract content to become visible
        WebDriverWait(section, 10).until(
            EC.visibility_of_element_located((By.XPATH, ".//div[contains(@class, 'issue-item__abstract__content') and contains(@class, 'collapse') and contains(@class, 'show')]"))
        )
        
        # Extract the abstract content
        abstract_element = section.find_element(By.XPATH, ".//div[contains(@class, 'issue-item__abstract__content') and contains(@class, 'collapse') and contains(@class, 'show')]")
        abstract_text = abstract_element.text
    except Exception as e:
        print(f"Error processing section for abstract. Error: {e}")
        abstract_text = "Abstract not available"

    # Extract titles, authors, publish dates, DOI URLs, and affiliations
    titles = section.find_elements(By.CLASS_NAME, 'issue-item__heading')
    authors = section.find_elements(By.CLASS_NAME, 'issue-item__authors')
    title_links = section.find_elements(By.CLASS_NAME, 'issue-item__title')
    doi_urls = [link.find_element(By.TAG_NAME, 'a').get_attribute('href') for link in title_links]
    
    # Extract publish date using the div with the class 'issue-item__header'
    try:
        header = section.find_element(By.CLASS_NAME, 'issue-item__header')
        publish_date = header.find_element(By.XPATH, ".//span[.//text()[contains(., 'First published')]]").text
    except Exception as e:
        print(f"Error processing publish date. Error: {e}")
        publish_date = "Publish Date not available"

    # Gather and store data
    for j, title in enumerate(titles):
        author_element = authors[j] if j < len(authors) else None
        orcids = []
        affiliations_text = ''

        if author_element:
            # Extract ORCID IDs and affiliations
            orcid_elements = author_element.find_elements(By.CLASS_NAME, 'orcid-id')
            orcids = [orcid.get_attribute('href') for orcid in orcid_elements]
            
            affiliations = author_element.find_elements(By.CLASS_NAME, 'author-affiliation')  # Adjust class name as needed
            affiliations_text = ', '.join([affil.text for affil in affiliations])
            
            author_text = author_element.text
        else:
            author_text = 'Authors not available'

        row = {
            'Title': title.text,
            'Authors': author_text,
            'Affiliations-ORCID IDs': ', '.join(orcids),
            'Publish Date': publish_date,
            'DOI URL': doi_urls[j] if j < len(doi_urls) else 'DOI URL not available',
            'Abstract': abstract_text
        }
        data.append(row)

# Create a DataFrame from the data
df = pd.DataFrame(data)

#Neglect below error - this is because final article doesn't have proper information to scrap from website

Error processing section for abstract. Error: Message: 
Stacktrace:
0   chromedriver                        0x0000000102c11024 cxxbridge1$str$ptr + 1887276
1   chromedriver                        0x0000000102c09700 cxxbridge1$str$ptr + 1856264
2   chromedriver                        0x000000010281882c cxxbridge1$string$len + 88524
3   chromedriver                        0x000000010285c834 cxxbridge1$string$len + 367060
4   chromedriver                        0x0000000102852e38 cxxbridge1$string$len + 327640
5   chromedriver                        0x000000010289448c cxxbridge1$string$len + 595500
6   chromedriver                        0x0000000102851474 cxxbridge1$string$len + 321044
7   chromedriver                        0x00000001028520e4 cxxbridge1$string$len + 324228
8   chromedriver                        0x0000000102bd8a08 cxxbridge1$str$ptr + 1656336
9   chromedriver                        0x0000000102bdd464 cxxbridge1$str$ptr + 1675372
10  chromedriver                        0

,Title,Authors,Affiliations-ORCID IDs,Publish Date,DOI URL,Abstract
0,The Caring Machine: Feeling AI for Customer Care,Ming-Hui Huang\nRoland T. Rust,"https://orcid.org/0000-0001-8954-3172, https:/...","First published March 22, 2024",https://journals.sagepub.com/doi/abs/10.1177/0...,Abstract\nCustomer care is important for its r...
1,Consequences of Marketing Asset Accountability...,Peter Guenther\nMiriam Guenther\nBryan A. Luka...,https://orcid.org/0000-0001-6033-8466,"First published April 22, 2024",https://journals.sagepub.com/doi/abs/10.1177/0...,Abstract\nMarketing scholars have extensively ...
2,Developing Strengths or Remedying Weaknesses? ...,Qihui Chen\nYajin Wang\nYing Zhang,"https://orcid.org/0009-0005-0623-6385, https:/...","First published May 6, 2024",https://journals.sagepub.com/doi/abs/10.1177/0...,Abstract\nDoes parents’ status motivation affe...
3,More Likely to Pay but Less Engaged: The Effec...,Joy Lu\nEric T. Bradlow\nJ. Wesley Hutchinson,https://orcid.org/0000-0001-7132-7105,"First published March 20, 2024",https://journals.sagepub.com/doi/abs/10.1177/0...,Abstract\nFollowing trends in entertainment st...
4,"So, Sue Me…If You Can! How Legal Changes Dimin...",Arvid O.I. Hoffmann\nChee S. Cheong\nHoàng-Lon...,"https://orcid.org/0000-0003-4148-5078, https:/...","First published April 7, 2024",https://journals.sagepub.com/doi/abs/10.1177/0...,Abstract\nResearch examining the antecedents o...


In [104]:
df.to_excel('/Users/trsaivarun/Desktop/c_py:R/assignment-2/output-final.xlsx', sheet_name='sheet1', index=False)


In [105]:
# saving as csv - according to UTF-8 encodings
df['Abstract'] = df['Abstract'].str.replace('\n', ' ', regex=False)
df['Authors'] = df['Authors'].str.replace('\n', ', ', regex=False)
df.to_csv('/Users/trsaivarun/Desktop/c_py:R/assignment-2/output-final.csv', index=False, sep='\t', encoding='utf-8-sig')
